In [1]:
import requests
import json
import pandas as pd

In [2]:
with open("discord_logs.json") as f:  # Open the json file as f
    logs = json.load(f)               # set the file to a variable

In [3]:
# Removing the un-needed strings
def the_event(string):                                     
    bad_chars = [';', ':', '!', "*", "/",'`','.','@']              # Initialized the special characters to a list
    test_string = ''.join(i for i in string if not i in bad_chars) # Remove the speacial characters from the list
    words = test_string.split()                           # Split each word in the list
    return words[1]                                       # Take the second word in the string

In [4]:
# Gives the Name, Event, Time at the position
def get_name(list_num):
    log = {}                                              # Initialized dictionary
    log["name"] = list_num["embeds"][0]["author"]["name"] # Get the name description from the Data
    log["event"] = list_num["embeds"][0]["description"]   # Get the event description from the Data
    log["time"] = list_num["embeds"][0]["timestamp"]      # Get the time description from the Data
    return log                                            # Return log

In [5]:
def list_diction(my_log):
        # The unwanted string in the dictionary-list
    unwanted_1= "**@Unknown joined the server.**"
    unwanted_2 = ":writing_hand: **@Unknown has been updated.**"
    unwanted_3 = '**@Unknown left.**'
    updated = []                          # Initialized empty list
    
    event_list = []                       # Initialized empty list
    for i in my_log["messages"]:          # Stepping through the data
        event_list.append(get_name(i))    # Add the name, event, and time to the event_list

    for j in event_list:                       # Stepping through the event_list
        if (j["event"] != unwanted_1):         # If the string does not match the unwanted_1 string
            if (j["event"] != unwanted_2):     # If the string does not match the unwanted_2 string
                if (j["event"] != unwanted_3): # If the string does not match the unwanted_3 string
                    updated.append(j)          # Add the data to the list, updated
    return updated                             # Return the list updated

In [6]:
def get_event(the_list):                       # Get the events "joined" and "left"
    left = "left"                              # Initialized the variable left to a string
    joined = "joined"                          # Initialized the variable joined to a string
    new_list = []                              # Initialized an empty list
    for ii in the_list:                        # Stepping through the the_list
        ii["event"] = the_event(ii["event"])   # Remove the unwanted symbols and characters from the string
    
    for i in the_list:                         # Stepping through the_list
        if (i['event'] == left) or (i["event"] == joined): # If the string is equal to the variables left and joined
            new_list.append(i)                             # Add it to the list new_list
    return new_list                                        # Return the new_list

# Part 1

In [7]:
data_frame = get_event(list_diction(logs))             # Assign the variable to the Data
df = pd.DataFrame(data_frame)                          # Create a Dataframe

df["time"] = pd.to_datetime(df["time"]).dt.tz_convert("America/New_York")# Convert the time in the DataFrame
df

,name,event,time
0,Jamya,joined,2021-01-25 14:31:13.449000-05:00
1,Jamya,left,2021-01-25 14:31:17.203000-05:00
2,Jamya,joined,2021-01-25 14:34:37.699000-05:00
3,Jamya,left,2021-01-25 14:34:41.426000-05:00
4,Jamya,joined,2021-01-25 15:56:06.055000-05:00
...,...,...,...
627,Jasmin,left,2021-03-27 12:43:51.750000-04:00
628,Ariana,joined,2021-03-28 15:02:10.440000-04:00
629,Kathy,joined,2021-03-28 15:02:10.899000-04:00
630,Kathy,left,2021-03-28 15:20:37.529000-04:00


In [8]:
# The unique names
def unique_names(the_list):
    list_names = []                      # Initialize empty list

    for i in the_list:                   # Stepping through the_list
        if i["name"] not in list_names:  # If name is not in list_names
            list_names.append(i["name"]) # Add it to list_names
    return list_names                    # Return list_names

In [9]:
def get_log(NAME,log_list):
    diction = {}                        # Initialize empty dictionary
    my_joined_list = []                 # Initialize empty list
    my_left_list = []                   # Initialize empty list
    for the in log_list:                # Stepping through the Data
        if the["name"]== NAME and the["event"]=="joined": # If the data is equal to the name and "joined"
            my_joined_list.append(the["time"])            # Add it to the list called, my_joined_list
        if the["name"]== NAME and the["event"]=="left":   # If the data is equal to the name and "left"
            my_left_list.append(the["time"])              # Add it to the list called, my_left_list
      
        
    
    my_new_df = pd.DataFrame({  # Created a Data Frame
    "name": NAME,               # Assign "name" to the names in the list
    "Start":my_joined_list,     # Assign "Start" to the my_joined_list
    "End": my_left_list         # Assign "End" to the my_left_list
    })

    # Converting the time in the list and getting the difference between Start and End 
    my_new_df["Start"] = pd.to_datetime(my_new_df["Start"]).dt.tz_convert("America/New_York")
    my_new_df["End"] = pd.to_datetime(my_new_df["End"]).dt.tz_convert("America/New_York")
    my_new_df["duration"] = my_new_df["End"] - my_new_df["Start"]

    diction["name"] = NAME                                   # Assign the "name" to the person name
    diction["time_in_channel"] = my_new_df["duration"].sum() # Assign "time_in_channel" to the 
                                                             #                total time on discord
    return diction    # Return diction


In [10]:
def data_log_list(names,log_list):      # 
    log = []                            # Initialized an empty list
    for i in names:                     # Stepping through the names
        log.append(get_log(i,log_list)) # Add the name and duration of a person to the list 
    return log                          # Return the log

In [11]:
def time_spent_on_discord(my_log):         # 
    list_of_people = list_diction(my_log)  # Getting the list of people
    updated_event_list = get_event(list_of_people) # Getting the updated events of people
    unique_ind = unique_names(updated_event_list)  # Getting the unique names from the list
    data = data_log_list(unique_ind,updated_event_list)  # Getting the list of people
                                           #        and the time spent on Discord
    
    df = pd.DataFrame(data)                  # Creating the Data frame
    dff = df.sort_values(by="time_in_channel", ascending=True) # Sorting the list by "time_in_channel"
   
    return dff                             # Return dff

# Part 2

In [12]:
dataframe = time_spent_on_discord(logs).reset_index(drop = True)  
dataframe.tail(5)                                                 

,name,time_in_channel
25,Ariana,0 days 17:37:12.113000
26,Jamya,1 days 01:12:34.347000
27,Kathy,1 days 03:15:31.871000
28,Edward,1 days 19:39:40.461000
29,Abagail,3 days 22:22:09.153000
